In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm
import joblib
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from pandas import Series, DataFrame
import warnings
warnings.filterwarnings('ignore')
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge,RidgeCV#线性回归模型的岭回归
from datetime import datetime
# from sklearn.externals import joblib
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import RFE
from sklearn.preprocessing import scale
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier     #随机森林用于分类
from sklearn.ensemble import RandomForestRegressor as RFR      #随机森林用于回归
from sklearn.model_selection import train_test_split           #划分训练集与测试集
from sklearn import metrics    
from sklearn.metrics import r2_score              #用于模型拟合优度评估
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from scipy.stats import pearsonr
pd.options.display.max_rows = None
import xgboost as xgb

import rdkit
import pandas as pd
import numpy as np
from collections import Counter

from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from mordred import Calculator, descriptors
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)



In [2]:
import joblib

GBRT_model = joblib.load("Model/Cp-GBRT-0.969.pkl")
GBRT_out= pd.read_excel("Out_data/Cp-GBRT-0.969.xlsx")



In [3]:
#molecule1
mols = Chem.SDMolSupplier("75-28-5-3d.sdf",strictParsing=False)
# Temperature（K）
Temperature = 303.65
# Cp(J/mol·K)
Cp = 142.970

In [4]:
# Molecular descriptor computation
calc = Calculator(descriptors, ignore_3D=True)
Mordred_description = []
Rdkit_description = [x[0] for x in Descriptors._descList]
for i in calc.descriptors:
    Mordred_description.append(i.__str__())
for i in Mordred_description:
    if i in Rdkit_description:
        Rdkit_description.remove(i)
        
Molecular_descriptor = []

descriptor_calculator = MoleculeDescriptors.MolecularDescriptorCalculator(Rdkit_description)
j =0
for i in mols:
    Calculator_descript = pd.DataFrame(calc.pandas([i]))
    rdkit_descriptors = pd.DataFrame([descriptor_calculator.CalcDescriptors(i)],columns=Rdkit_description)
    Calculator_descript = Calculator_descript.join(rdkit_descriptors)
    Molecular_descriptor.append(Calculator_descript)
    j+=1
    print(j)
    
a = Molecular_descriptor[0]
for i in Molecular_descriptor[1:]:
    a = a.append(i)
a = a.reset_index(drop=True)
# Delete a value that failed to be computed
a = a.drop(labels=a.dtypes[a.dtypes == "object"].index,axis=1)
a = a.loc[:,["ATS0s","ATS0pe","ATSC0m","NumValenceElectrons","Kappa2"]]
a.insert(0, "Temperature",Temperature)
b = a.copy()
a


100%|██████████| 1/1 [00:00<00:00,  4.84it/s]


1


,Temperature,ATS0s,ATS0pe,ATSC0m,NumValenceElectrons,Kappa2
0,303.65,23.777778,74.41,345.902883,26,1.333333


In [9]:
# Data standardization
GBRT_train = GBRT_out.iloc[:int(len(GBRT_out)*0.7)-1,4:]

GBRT_stand_scaler = StandardScaler()
GBRT_train_std = GBRT_stand_scaler.fit_transform(GBRT_train)
GBRT_test_std_1 = GBRT_stand_scaler.transform(a) 


predict_value = GBRT_model.predict(GBRT_test_std_1)
print("predict_value",predict_value[0])
# r2 = r2_score(Cp, GBRT_model.predict(predict_value))
# print("r2",r2)



predict_value 134.2806661115856
